In [1]:
#import
from bs4 import BeautifulSoup
import requests
import numpy as np
import json

In [2]:
valid = "/Users/anhnguyen/Desktop/research/scraping_Python/text_files/valid_march.txt"
file = '/Users/anhnguyen/Desktop/research/scraping_Python/text_files/json.txt'

### Helper Function

In [3]:
def write_file(str, file_name):
    with open(file_name, "a") as text_file:
        text_file.write(str)
        
def read_txt(txt_file):
    links = []
    count = 0
    with open(txt_file) as f:
        for line in f:   
            count +=1
            links += [line.rstrip()]
    return links, count
def readtxt(txt_file):
    links = []
    count = 0
    with open(txt_file) as f:
        for line in f:   
            elems = line.split('\t')
            #elem is a tuple (name, link)
            elem =  (elems[0].rstrip(), elems[1].rstrip())
            count +=1
            links += [elem]
    return links, count

In [4]:
links, count = readtxt(valid)
print(links[0][1])

https://www.richland2.org/charterhigh/


In [5]:
url = 'https://www.polk.edu/lakeland-gateway-to-college-high-school/'
def parse_data(url):
    data  = requests.get(url).text
    soup = BeautifulSoup(data, "lxml")
    hrefs = soup('a')
    paragraphs = soup('p')
    #only want unique hrefs
    links = []
    for each in hrefs:
        links.append(each.get('href'))
    links = set(links)
    return links, paragraphs

In [6]:

# print(links)

In [7]:
urlkey = ['about', 'department', 'home', 'community', 'life', 'curriculum', 'hist', 'phil', 'academics', 'moral', 'focus', 'beliefs', 'program', 'resources', 'mission', 'value', 'core']
urlkey = [s.upper() for s in urlkey]
#now parse through all of those links based on the key word search,
#and get a new list which contains the url's we actually want
final_list = []
for k in links:
    try:
        j = k.upper()
        for i in range(len(urlkey)):
            if isinstance(j, list) and len(j) == 1:
                j = j[0].split(urlkey[i])
            elif isinstance(j, list) and len(j) > 1:
                if requests.get(url + k):
                    final_list.append(k)
                    break
            else:
                j = j.split(urlkey[i])
    except:
        pass

final_list += ['/home']
for i in range(len(final_list)):
    each = final_list[i].upper()
    k, m, n, o = each.split('FREQ'), each.split('ADMI'), each.split('TRUSTE'), each.split('ALUM')
    if len(k) > 1 or len(m) > 1 or len(n) > 1 or len(o) > 1:
        final_list[i] = '/home'
final_list = set(final_list)
print(final_list)
print(len(final_list))

{'/home', 'https://www.polk.edu/lakeland-gateway-to-college-high-school/student-resources/', 'https://www.polk.edu/lakeland-gateway-to-college-high-school/about-us/', 'https://www.polk.edu/about/', '/people/department/lakeland-gateway-to-college-high-school/', 'https://www.polk.edu/academics/', 'https://www.polk.edu/student-life/', 'https://www.polk.edu/community/'}
8


In [8]:
#create a dictionary to for us to store our relevant texts into into 
#their respective categories
school = {'mission' : [], 'curriculum': [], 'philosophy' : [], 'history' : [], 'target' : [], 'resources' : [], 'organizational_factors' : []}
school

{'curriculum': [],
 'history': [],
 'mission': [],
 'organizational_factors': [],
 'philosophy': [],
 'resources': [],
 'target': []}

In [9]:
#keywords

k = [str(x) for x in np.arange(1900,2012)]
idk = ['advantage']
mission = ['mission',' vision ', 'our purpose', 'cause', 'objectives', 'vision:', 'mission:']
curriculum = ['curriculum', 'program', 'methods', 'pedagogy', 'approach', 'model',  'academics', 'degree']
philosophy = ['philosophy','beliefs', 'principles', 'creed', 'values',  'moral']
history = ['history', 'story', 'background', 'founded', 'established'] + k
target = ['target','gifted', 'at-risk', 'ethnic background', 'target population', 'target audience']
resources = ['resources', 'ESL', 'tutoring', 'after-school programs', 'available resources', 'services', 'opportunities', 'opportunity']
orgfactor = ['organizational_factors', 'statistics', 'API', 'environment', 'ratio', 'average', 'female', 'fund', 'community'  ]
keywords = [(mission[0], mission), (curriculum[0], curriculum), (philosophy[0], philosophy), (history[0], history), ( target[0],target) , (resources[0], resources), (orgfactor[0], orgfactor) ]
for i in range(0,len(keywords)):
    keywords[i] = (keywords[i][0], [s.upper() for s in keywords[i][1]])
keywords[0][1]

['MISSION',
 ' VISION ',
 'OUR PURPOSE',
 'CAUSE',
 'OBJECTIVES',
 'VISION:',
 'MISSION:']

In [10]:
#create a mapper function to map the text which we parse through into 
#its relevant category in the dictionary

def mapper(txt):
#     print(txt.upper())
    text = txt.upper()
    for i in range(len(keywords)):
        update_with_keyword(keywords[i], txt)
        
def update_with_keyword(keyword, txt):
    text1 = txt.upper()
#     print(keyword[1])
    for i in range(len(keyword[1])):
        if not isinstance(text1, list):
            text1 = text1.split(keyword[1][i])
        elif len(text1) == 1:
            text1 = text1[0].split(keyword[1][i])
        if len(text1) > 1:
#             print(keyword[0])
            school[keyword[0]].append(txt)
            break

In [11]:
for link in final_list:
    q = requests.get(url + link)
    k = q.text
    s = BeautifulSoup(k, "lxml")
    j = s.find_all('p')
    for lin in j:
        k = str(lin)
        if lin.get('class') == None and lin.get('a') == None and len(k.split('|')) == 1:
            mapper(' ' + lin.get_text())

In [12]:

updated_school = dict((k, set(v)) for k, v in school.items())

In [13]:
print(updated_school)

{'resources': {' No matter if you’re into the environment or salsa dancing, there is a Polk State student club or organization for you. The College offers dozens of student life opportunities — all you have to do is join. Don’t see what you’re looking for? Contact your campus Student Activities and Leadership Office (SALO) for information on how to start a new club.', ' This public charter school represents an exciting new opportunity for local area students who have earned 45% of the credits required for high school graduation, providing students with maximum flexibility and opportunity for success. The school was funded in part through a grant from the Gateway to College National Network.', ' Polk State’s advantages include small class sizes; hands-on learning; dedicated faculty; day and night classes; online options; affordable tuition; excellent financial aid; a diverse community and vibrant student life, including fine arts opportunities, clubs, and intercollegiate athletics.', ' 

In [14]:
print(updated_school['mission'])

{' Vision Statement:\xa0All GHS enrollees will graduate from high school and college and become self-actualized citizens and successful leaders in their communities.', ' The mission of Polk State Lakeland Gateway to College Collegiate High School, is to provide an\xa0academic charter alternative for juniors and seniors transitioning to college.', ' Polk State College is proud to be part of FUSE, a program in partnership with the University of South Florida that provides\xa0eligible Polk State students with guaranteed admission to a USF System institution upon the successful completion of an associate in arts degree. This program is for students who are starting at Polk State just out of high school, and those that applied to\xa0a USF System institution and were not immediately admitted.\xa0Students who participate in FUSE\xa0will:'}


In [15]:

with open(file, 'w') as outfile:
    json.dump(school, outfile)